In [1]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import joblib

loaded_pipeline = joblib.load('svm_model.joblib')

scaler = loaded_pipeline.named_steps['standardscaler']
svm_classifier = loaded_pipeline.named_steps['svc']

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

app = Flask(__name__)

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return 'No file part'

    file = request.files['file']
    
    # Do something with the file, e.g., save it to the server
    file.save('uploaded_file.csv')

    return 'File uploaded successfully'

@app.route('/predict', methods=['POST'])
def predict():
    received_data = pd.read_csv('uploaded_file.csv')
    X_new = received_data
    X_new_standardized = scaler.transform(X_new)
    prediction = svm_classifier.predict(X_new_standardized).astype(int)

    confidence_scores = svm_classifier.decision_function(X_new_standardized)
    confidence_percentage = round(float(sigmoid(confidence_scores) * 100),2)

    return jsonify({'prediction': prediction.tolist(), 'confidence': confidence_percentage})

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2024-03-24 12:07:04,095] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "/home/animesh/miniconda3/lib/python3.10/site-packages/flask/app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/animesh/miniconda3/lib/python3.10/site-packages/flask/app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/animesh/miniconda3/lib/python3.10/site-packages/flask/app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/animesh/miniconda3/lib/python3.10/site-packages/flask/app.py", line 1469, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/tmp/ipykernel_6247/1750172450.py", line 36, in predict
    confidence_percentage = round(float(sigmoid(confidence_scores) * 100),2)
TypeError: only size-1 arrays can be converted to Python scalars
127.0.

In [9]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import joblib
from flask_cors import CORS

loaded_pipeline = joblib.load('svm_model.joblib')

scaler = loaded_pipeline.named_steps['standardscaler']
svm_classifier = loaded_pipeline.named_steps['svc']

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

app = Flask(__name__)
CORS(app)

@app.route('/predict', methods=['POST'])
def predict():
    received_data = pd.read_csv('uploaded_file.csv')
    X_new = received_data
    X_new_standardized = scaler.transform(X_new)
    prediction = svm_classifier.predict(X_new_standardized).astype(int)
    
    confidence_scores = svm_classifier.decision_function(X_new_standardized)
    # print(type(confidence_scores))
    confidence_percentages = np.array([round(float(sigmoid(i) * 100), 2) for i in confidence_scores])

    return jsonify({'prediction': prediction.tolist(), 'confidence': confidence_percentages.tolist()})

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [24/Mar/2024 12:25:05] "POST /predict HTTP/1.1" 200 -
